In [2]:
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


class GoogleMapScanner(object):

    def __init__(self, browser_size=(1024, 800), timeout=5):
        # Setup the browser configuration
        self.base_url = 'https://www.google.com/maps/place/'
        self.timeout = timeout
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        self.browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
        self.browser.set_window_size(*browser_size)

    def generate_lat_long_coordinates(self, base_coordinate, lat_steps=(0,), long_steps=(0,)):
        """Compute latitudes and longitudes for a given base coordinate and latitude and longitude step"""
        coordinates = [(base_coordinate[0] + lat_step, base_coordinate[1] + long_step) for lat_step in lat_steps for
                       long_step in long_steps]
        return coordinates

    def clean_map(self, remove_search_pane=False):
        """Remove overlays from Google map"""
        # Remove omnibox
        js_string = "var element = document.getElementById(\"omnibox-container\");element.remove();"
        self.browser.execute_script(js_string)
        # Remove username and icons
        js_string = "var element = document.getElementById(\"vasquette\");element.remove();"
        self.browser.execute_script(js_string)
        # Remove bottom scaling bar
        js_string = "var element = document.getElementsByClassName(\"app-viewcard-strip\");element[0].remove();"
        self.browser.execute_script(js_string)
        # Remove attributions at the bottom
        js_string = "var element = document.getElementsByClassName(\"scene-footer-container\");element[0].remove();"
        self.browser.execute_script(js_string)
        if remove_search_pane:
            # Remove search pane
            js_string = "var element = document.getElementsByClassName(\"widget-pane-visible\");element[0].remove();"
            self.browser.execute_script(js_string)

    def scan_coordinates(self, base_coordinate, lat_steps, long_steps, zoom=200, output_dir='C:/Temp/'):
        """Perform a scan of a grid with bottom left defined by the base_coordinate"""
        lat_long = self.generate_lat_long_coordinates(base_coordinate, lat_steps, long_steps)
        urls = [self.base_url + f'@{lat},{long},{zoom}m/data=!3m1!1e3' for lat, long in lat_long]
        self.scan_urls(urls, output_dir)

    def scan_addresses(self, addresses, output_dir='C:/Temp/'):
        """Perform a scan of a grid with bottom left defined by the base_coordinate"""
        urls = [self.base_url + address.replace(' ', '+') + '/data=!3m1!1e3' for address in addresses]
        self.scan_urls(urls, output_dir, remove_search_pane=True)

    def get_image_label(self, url):
        """Convert label to human readable label"""
        label = url.replace(self.base_url, '').split('/')[0]
        return label

    def scan_urls(self, urls, output_dir, remove_search_pane=False):
        """Scan GoogleMaps and save images for each of the urls provided"""
        if type(urls) is str:
            urls = list(urls)
        for url in urls:
            print(f'Scanning: {url}')
            self.browser.get(url)
            label = self.get_image_label(url)
            try:
                element_present = EC.presence_of_element_located((By.CLASS_NAME, 'widget-offscreen-pegman-marker'))
                WebDriverWait(self.browser, self.timeout).until(element_present)
                self.clean_map(remove_search_pane)
                self.browser.save_screenshot(output_dir + f"{label}.png")

            except TimeoutException:
                print("Timed out")


if __name__ == '__main__':

    scanner = GoogleMapScanner()

    # Scanning GPS coordinates
    base_coordinate = (51.5139496, -0.0882675)
    lat_steps = np.linspace(0, 6e-3, 3)
    long_steps = np.linspace(0, 6e-3, 3)
    scanner.scan_coordinates(base_coordinate, lat_steps, long_steps)

    # Scanning addresses
    addresses = ['1 Threadneedle St, London EC2R 8AH', '100 Threadneedle St, London EC2R 8AH']
    scanner.scan_addresses(addresses)


ModuleNotFoundError: No module named 'selenium'